In [4]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T
import torchvision.datasets as D
import torch.nn.functional as F
import numpy as np
import pickle
from matplotlib import pyplot as plt
from utils.utils import MyDataset, show_image, visualise_output
import sys
sys.path.append("..")
from networks.autoencoder import FireAutoencoder
from networks.autoencoder_reward import FireAutoencoder_reward
import matplotlib
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import scipy

In [5]:
dataset = MyDataset(root='../data/complete_random/homo_2/Sub20x20_full_grid_.pkl',
                             tform=lambda x: torch.from_numpy(x, dtype=torch.float))

In [30]:
latent_dims = 256
capacity = latent_dims//2 
input_size = 20
epochs = 100
sigmoid = False
net = FireAutoencoder(capacity, input_size, latent_dims, sigmoid)
net.load_state_dict(torch.load(f'weights/v1/homo_2_sub20x20_latent={latent_dims}_capacity={capacity}_{epochs}.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [31]:
train_dataset, validation_dataset, test_dataset =torch.utils.data.random_split(dataset, [0.9, 0.05, 0.05])

In [32]:
batch = 16
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch, shuffle=False)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch, shuffle=False)
test_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch, shuffle=False)

In [33]:
full_loader  = torch.utils.data.DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=False)

In [34]:
all_images, all_r = next(iter(full_loader))

In [35]:
embeddings = net.encode(all_images)

In [36]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T
import torchvision.datasets as D
import torch.nn.functional as F
import numpy as np

class ANN(nn.Module):
    def __init__(self, latent_dims):
        super(ANN, self).__init__()
   
        # Reward predictor:
        self.fc_r1 = nn.Linear(in_features=latent_dims, out_features=128)
        self.fc_r2 = nn.Linear(in_features=128, out_features=64)
        self.fc_r3 = nn.Linear(in_features=64, out_features=32)
        self.fc_r4 = nn.Linear(in_features=32, out_features=1)

    
    def forward(self, x):
        h1 = F.relu(self.fc_r1(x))
        h2 = F.relu(self.fc_r2(h1))
        h3 = F.relu(self.fc_r3(h2))
        reward = self.fc_r4(h3)
        return reward


In [37]:
reward_ann = ANN(latent_dims)

In [38]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(reward_ann.parameters(), lr = 0.0001)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
reward_ann.to(device)
epochs = 50
training_loss = []
validation_loss = []
for epoch in range(epochs):
    n = 0
    m = 0
    epoch_loss = 0
    val_epoch_loss = 0
    for x, r in train_loader:
        embedding = net.encode(x)
        r = r.to(device)
        output = reward_ann(embedding)
        loss = criterion(output.squeeze(), r)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        n += 1
    
    for y, r in validation_loader:
        embedding = net.encode(y)
        r = r.to(device)
        output = reward_ann(embedding)
        val_loss = criterion(output.squeeze(),r)
        optimizer.zero_grad()
        val_epoch_loss += val_loss.item()
        m+=1
    training_loss.append(epoch_loss/n)
    validation_loss.append(val_epoch_loss/m)
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, loss.item()))
    print('Epoch [{}/{}], Validation Loss: {:.4f}'.format(epoch+1, epochs, val_loss.item()))

Epoch [1/50], Loss: 115325.7188
Epoch [1/50], Validation Loss: 138913.9531
Epoch [2/50], Loss: 31163.5449
Epoch [2/50], Validation Loss: 49147.2539
Epoch [3/50], Loss: 19544.2383
Epoch [3/50], Validation Loss: 25858.3164
Epoch [4/50], Loss: 10994.1006
Epoch [4/50], Validation Loss: 12698.6641
Epoch [5/50], Loss: 5946.4868
Epoch [5/50], Validation Loss: 6254.0693
Epoch [6/50], Loss: 3810.4800
Epoch [6/50], Validation Loss: 3885.4492
Epoch [7/50], Loss: 3213.9839
Epoch [7/50], Validation Loss: 3378.4434
Epoch [8/50], Loss: 3086.2876
Epoch [8/50], Validation Loss: 3319.9885
Epoch [9/50], Loss: 3057.2861
Epoch [9/50], Validation Loss: 3311.9207
Epoch [10/50], Loss: 3048.7009
Epoch [10/50], Validation Loss: 3308.2202
Epoch [11/50], Loss: 3045.0935
Epoch [11/50], Validation Loss: 3304.8975
Epoch [12/50], Loss: 3043.3960
Epoch [12/50], Validation Loss: 3302.8699
Epoch [13/50], Loss: 3042.1089
Epoch [13/50], Validation Loss: 3302.2993
Epoch [14/50], Loss: 3041.1543
Epoch [14/50], Validation Lo

In [ ]:
plt.ion()
fig = plt.figure()
plt.plot(training_loss[1:], label='training loss')
plt.plot(validation_loss[1:], label='validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [ ]:
embeddings = net.encode(all_images)

In [ ]:
with torch.no_grad():
    rewards = reward_ann(embeddings)

In [ ]:
bins = np.arange(-1020, -500, 10)
plt.hist(rewards.squeeze().numpy(), bins=bins, align='left')
plt.title('Distribución de las recompensas predecidas')
plt.xlabel('Recompensa')
plt.ylabel('Frecuencia')
#plt.savefig(f"linear_regresion/reward_classes_distr_{latent_dims}_{epochs}_sigmoid={sigmoid}.png.png")
plt.show()

In [ ]:
def ann(x):
    return reward_ann(torch.from_numpy(x).float()).detach().numpy()

In [ ]:
res = scipy.optimize.minimize(ann, x0=np.zeros(latent_dims))

In [ ]:
res

In [ ]:
minimum = torch.from_numpy(res.x)

In [ ]:
net.float()

In [ ]:
solution = net.decode(minimum.float().unsqueeze(0))

In [ ]:
solution

In [ ]:
if sigmoid:
    solution[solution>=0.5] = 1
    solution[solution<=0.5] = 0
else:
    solution[solution>0] = 1

In [ ]:
with torch.no_grad():
        plt.title('Reconstrucción del mínimo')
        plt.imshow(solution[0][0].numpy())
        #plt.savefig(f"linear_regresion/minimum_decoding_{latent_dims}_{epochs}_sigmoid={sigmoid}.png")
        plt.colorbar()
        plt.show()

In [ ]:
with torch.no_grad():
    print(repr(solution[0][0].numpy()))